In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from math import ceil
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
import time

In [2]:
url = 'https://www.naukri.com/bank-jobs?xt=catsrch&qf[]=6'

In [3]:
option = webdriver.ChromeOptions()
# option.add_argument('--headless')
option.add_argument("--disable-web-security")
option.add_argument("--disable-site-isolation-trials")
option.add_argument("--disable-features=SameSiteByDefaultCookies")
option.add_argument('--disable-gpu')
option.add_argument('--disable-extensions')
option.add_argument('--no-sandbox')

browser = webdriver.Chrome(options = option)
browser.get(url)
browser.maximize_window()
sleep(5)

In [ ]:
def extract_info(browser):
    
    job_list = []
    
    job_elements = browser.find_elements(By.CLASS_NAME, "srp-jobtuple-wrapper")
    if len(job_elements) == 0:
        print("Elements not found")
        
    # Loop through each job element and extract the required information
    for job in job_elements:
        
        # Create a dictionary to store the current job's information
        job_info = {}

        # Extract job title
        job_info['Job Title'] = job.find_element(By.CLASS_NAME, "title").text

        # Extract company name
        job_info['Company Name'] = job.find_element(By.CLASS_NAME, "comp-name").text

        try:
            experience_walk_in_date_element = job.find_element("css selector", "span.ni-job-tuple-icon.ni-job-tuple-icon-srp-experience.exp")
            experience = experience_walk_in_date_element.text
            
        except NoSuchElementException:
            experience = ""
            
        try:
            experience_walk_in_date_element = job.find_element("css selector", "span.ni-job-tuple-icon.ni-job-tuple-icon-srp-date.exp")
            walk_in = experience_walk_in_date_element.text
            
        except NoSuchElementException:
            walk_in = ""
                      
                        
        job_info['Walk In Date'] = walk_in
        job_info['Experience'] = experience

        
        try:
            job_info['Salary'] = job.find_element(By.CLASS_NAME, "sal-wrap").text
        except:
            job_info['Salary'] = "Not disclosed"

        # Extract location
        parent_element = job.find_element("css selector", "span.ni-job-tuple-icon.ni-job-tuple-icon-srp-location.loc")
        job_info['Location'] = parent_element.find_element("xpath", "./span").text

        # Extract description
        job_info['Description'] = job.find_element(By.CLASS_NAME, "job-desc").text

        # Extract tags (skills)
        job_info['Tags'] = [tag.text for tag in job.find_elements(By.CLASS_NAME, "tag-li")]

        # Extract posting information (e.g., how many days ago it was posted)
        job_info['Posting Info'] = job.find_element(By.CLASS_NAME, "job-post-day").text

        job_info['Datetime Extracted'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        # Add the job_info dictionary to job_list
        job_list.append(job_info)

    return job_list  # Return the list containing all job dictionaries

In [ ]:
job_list = []
job_list += extract_info(browser)

i = 1
while url != '':

    pagination_div = browser.find_element(By.CLASS_NAME, 'styles_pagination__oIvXh')
    
    # Find all anchor tags within that div
    anchor_tags = pagination_div.find_elements(By.TAG_NAME, 'a')

    next_page = anchor_tags[-1]
    print(f'Link: {next_page.get_attribute("href")}, Text: {next_page.text}') 

    url = next_page.get_attribute("href")
    browser.get(url)

    sleep(3)

    print("Next page")
    
    job_list += extract_info(browser)

In [6]:
len(job_list)

460

In [7]:
import csv

csv_filename = 'jobs.csv'
if job_list:  
    headers = job_list[0].keys()
    
    with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
        
        writer = csv.DictWriter(file, fieldnames=headers)
        writer.writeheader()
        writer.writerows(job_list)

    print(f"Data successfully saved to {csv_filename}")
else:
    print("No data to write to CSV.")

Data successfully saved to jobs.csv
